In [1]:
# Cell 1: Import libraries + path + data reading & merging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from joblib import Parallel, delayed
import math
import os
import time
from scipy.stats import gaussian_kde, probplot, shapiro
import statsmodels.api as sm

start_time = time.time()

# ==============================================================
# 1. Paths & site info
# ==============================================================
site_info = [
    (r"E:\jupyter_data\Hapke\Submit\data\Original\ave_01_o.csv",
     r"E:\jupyter_data\Hapke\Submit\data\Original\shice_01.csv",
     r"E:\jupyter_data\Hapke\Submit\data\processed\Hapke_result",
     30.57, 0.0, 153.28, 59.31, 0.0, 261.17, "01"),
    (r"E:\jupyter_data\Hapke\Submit\data\Original\ave_03_o.csv",
     r"E:\jupyter_data\Hapke\Submit\data\Original\shice_03.csv",
     r"E:\jupyter_data\Hapke\Submit\data\processed\Hapke_result",
     37.42, 0.0, 130.25, 43.69, 0.0, 241.63, "03")
]

output_root = site_info[0][2]
os.makedirs(output_root, exist_ok=True)

# ==============================================================
# 2. Plot settings
# ==============================================================
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['font.size'] = 10
plt.rcParams['axes.unicode_minus'] = False

# ==============================================================
# 3. Hapke model functions
# ==============================================================
def cos_g(i, e, phi): return np.cos(i) * np.cos(e) + np.sin(i) * np.sin(e) * np.cos(phi)
def mu_0(i): return np.cos(i)
def mu(e): return np.cos(e)

def B_g(g, B_0, h):
    tan_half_g = np.tan(g / 2)
    return B_0 / (1 + (1 / h) * tan_half_g if tan_half_g != 0 else 1 + 1e-10)

def P_g(g, b, c):
    cos_g_val = np.cos(g)
    d1 = max(1 - 2 * b * cos_g_val + b**2, 1e-10)
    d2 = max(1 + 2 * b * cos_g_val + b**2, 1e-10)
    b = min(b, 0.99999)
    term1 = ((1 + c) / 2) * (1 - b**2) / (d1)**1.5
    term2 = ((1 - c) / 2) * (1 - b**2) / (d2)**1.5
    return term1 + term2

def H(mu, w): return (1 + 2 * mu) / (1 + 2 * mu * np.sqrt(1 - w))

def R_hapke(mu_0, mu, g, w, B_0, h, b, c):
    B = B_g(g, B_0, h)
    P = P_g(g, b, c)
    H0 = H(mu_0, w)
    H_mu = H(mu, w)
    return (w / (4 * np.pi * (mu_0 + mu))) * ((1 + B) * P + H0 * H_mu - 1)

# ==============================================================
# 4. Wavelength band definitions
# ==============================================================
wavelengths = [350 + (i-1)*4 for i in range(1, 165)]
selected_wavelengths = wavelengths[10:154]
band_range = (11, 154)
n_bands = band_range[1] - band_range[0] + 1

# ==============================================================
# 5. Read & merge data
# ==============================================================
all_ref_1a, all_ref_1b, all_tree_ids, all_geom = [], [], [], []

for idx, (inp_file, shice_file, _, i1_deg, e1_deg, phi1_deg,
          i2_deg, e2_deg, phi2_deg, prefix) in enumerate(site_info):
    shice_df = pd.read_csv(shice_file)
    valid_ids = shice_df['Tree_ID'].values
    df = pd.read_csv(inp_file)
    df = df[df['Tree_ID'].isin(valid_ids)].copy()
    df['Tree_ID'] = prefix + '_' + df['Tree_ID'].astype(str)

    start_band, end_band = band_range
    bands_1a = [f'1a_{350 + (i-1)*4}nm' for i in range(start_band, end_band + 1)]
    bands_1b = [f'1b_{350 + (i-1)*4}nm' for i in range(start_band, end_band + 1)]

    ref_1a = df[bands_1a].values
    ref_1b = df[bands_1b].values
    tree_ids = df['Tree_ID'].values

    all_ref_1a.append(ref_1a)
    all_ref_1b.append(ref_1b)
    all_tree_ids.extend(tree_ids)

    i1, e1, phi1 = map(math.radians, [i1_deg, e1_deg, phi1_deg])
    i2, e2, phi2 = map(math.radians, [i2_deg, e2_deg, phi2_deg])
    mu0_1 = mu_0(i1); mu_1 = mu(e1); g1 = np.arccos(cos_g(i1, e1, phi1))
    mu0_2 = mu_0(i2); mu_2 = mu(e2); g2 = np.arccos(cos_g(i2, e2, phi2))
    geom = (mu0_1, mu_1, g1, mu0_2, mu_2, g2)
    all_geom.extend([geom] * len(tree_ids))

actual_reflectance_1a = np.vstack(all_ref_1a)
actual_reflectance_1b = np.vstack(all_ref_1b)
tree_ids = np.array(all_tree_ids)
n_samples = actual_reflectance_1a.shape[0]

# Reindex tree_ids
new_tree_id = np.arange(1, n_samples + 1)
tree_id_map = pd.DataFrame({'Original_Tree_ID': tree_ids, 'New_Tree_ID': new_tree_id})

print(f"Data loaded. Total samples: {n_samples}")

Data loaded. Total samples: 75


In [2]:
# Cell 2: Optimization fitting + save parameters & fitted values
def optimize_sample_with_geom(sample_idx, actual_1a, actual_1b, geom_param):
    mu0_1, mu_1, g1, mu0_2, mu_2, g2 = geom_param
    def objective(params):
        w_vals = np.clip(params[:n_bands], 0, 1)
        B0 = np.clip(params[n_bands], 0, 1)
        h = np.clip(params[n_bands+1], 0.01, 1)
        b = np.clip(params[n_bands+2], 0, 1)
        c = np.clip(params[n_bands+3], -1, 1)
        model_1a = R_hapke(mu0_1, mu_1, g1, w_vals, B0, h, b, c)
        model_1b = R_hapke(mu0_2, mu_2, g2, w_vals, B0, h, b, c)
        return np.sum((model_1a - actual_1a)**2) + np.sum((model_1b - actual_1b)**2)
    init = np.concatenate([np.full(n_bands, 0.5), [1.0, 0.1, 0.5, 0.0]])
    bounds = [(0,1)]*n_bands + [(0,1), (0.01,1), (0,1), (-1,1)]
    res = minimize(objective, init, bounds=bounds, method='L-BFGS-B', options={'maxfun': 1_000_000, 'maxiter': 1_000_000})
    if not res.success:
        print(f"Warning: Sample {sample_idx} optimization failed: {res.message}")
    w = res.x[:n_bands]
    B0, h, b, c = res.x[n_bands:n_bands+4]
    fit_1a = R_hapke(mu0_1, mu_1, g1, w, B0, h, b, c)
    fit_1b = R_hapke(mu0_2, mu_2, g2, w, B0, h, b, c)
    return w, B0, h, b, c, fit_1a, fit_1b

print(f"Starting parallel fitting ({n_samples} samples)...")
results = Parallel(n_jobs=-1, verbose=10)(
    delayed(optimize_sample_with_geom)(i, actual_reflectance_1a[i], actual_reflectance_1b[i], all_geom[i])
    for i in range(n_samples)
)
w_est, B0_est, h_est, b_est, c_est, fit_1a_list, fit_1b_list = zip(*results)
w_est = list(w_est); B0_est = list(B0_est); h_est = list(h_est)
b_est = list(b_est); c_est = list(c_est)
fit_1a_list = list(fit_1a_list); fit_1b_list = list(fit_1b_list)

# Save parameters
w_all = np.column_stack(w_est).T
w_cols = {f'w_{wl}nm': w_all[:, i] for i, wl in enumerate(selected_wavelengths)}
params_df = pd.DataFrame({'New_Tree_ID': new_tree_id, 'Original_Tree_ID': tree_ids,
                          'B_0': B0_est, 'h': h_est, 'b': b_est, 'c': c_est})
params_df = pd.concat([params_df, pd.DataFrame(w_cols)], axis=1)
params_df.to_csv(os.path.join(output_root, "hapke_parameters_merged.csv"), index=False)

# Save fitted reflectance
fitted_cols = [pd.Series(new_tree_id, name='New_Tree_ID'), pd.Series(tree_ids, name='Original_Tree_ID')]
for i, wl in enumerate(selected_wavelengths):
    col_1a = [fit_1a_list[j][i] for j in range(n_samples)]
    col_1b = [fit_1b_list[j][i] for j in range(n_samples)]
    fitted_cols.append(pd.Series(col_1a, name=f'Fitted_1a_{wl}nm'))
    fitted_cols.append(pd.Series(col_1b, name=f'Fitted_1b_{wl}nm'))
pd.concat(fitted_cols, axis=1).to_csv(os.path.join(output_root, "merged_fitted_reflectance_390-962.csv"), index=False)

print("Fitting completed, parameters have been saved.")

Starting parallel fitting (75 samples)...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  28 out of  75 | elapsed:    4.6s remaining:    7.8s
[Parallel(n_jobs=-1)]: Done  36 out of  75 | elapsed:    5.8s remaining:    6.2s
[Parallel(n_jobs=-1)]: Done  44 out of  75 | elapsed:    6.4s remaining:    4.5s
[Parallel(n_jobs=-1)]: Done  52 out of  75 | elapsed:    7.6s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done  60 out of  75 | elapsed:    8.1s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done  68 out of  75 | elapsed:    8.4s remaining:    0.8s


Fitting completed, parameters have been saved.


[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:   13.5s finished
